# Chat with the system

Warning: This notebook may be outdated. The latest version is in the discord bot at `generator/story/converse.py`.


In [3]:
language = "french"
setting = "Post-apocalypse zombie"
first_message = "Vous vous trouvez dans une communauté isolée, barricadée et surveillée, où quelques dizaines de survivants comme vous ont trouvé refuge. Vous avez perdu des proches lors de l'épidémie et vous êtes déterminé à trouver un moyen de vaincre les Errants et de reconstruire votre monde. Vous avez reçu une mission, et vous savez que votre seul espoir de survie réside dans la découverte d'un remède contre l'épidémie. Vous devez partir à la recherche d'informations sur ce remède, mais pour cela, vous devrez quitter la sécurité relative de votre communauté et affronter les dangers du monde extérieur."

USE_LOCAL_LLM = False

In [26]:
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX
from langchain_fireworks import ChatFireworks
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.globals import set_verbose
from langchain.globals import set_debug
from langchain_core.output_parsers import JsonOutputParser
from typing import Optional


import getpass
import os

from dotenv import load_dotenv

load_dotenv()

# set_debug(True)
# set_verbose(True)

model = None
predictable_model = None

# Again, we used MLX for local inference on a Macbook Pro, but it was too slow.
# Ollama offers a much faster inference. However, Ollama has not been implemented
# in this notebook. It is implemented in the discord bot.

if USE_LOCAL_LLM:
    print("Loading model...")
    # Load model from huggingface, using the MLX framework to take advantage of Apple Silicon
    llm = MLXPipeline.from_model_id(
        "mlx-community/Meta-Llama-3.1-8B-Instruct-8bit",
        # "mlx-community/Meta-Llama-3.1-8B-Instruct-bf16",
        # "mlx-community/Qwen2.5-32B-Instruct-4bit", # Let's try using a larger model, see if that improves results
        # "mlx-community/Llama-3.2-3B-Instruct-8bit",
        pipeline_kwargs={"max_tokens": 2048, "temp": 0.2, "repetition_penalty": 1.2},
    )

    predictable_llm = MLXPipeline.from_model_id(
        "mlx-community/Meta-Llama-3.1-8B-Instruct-8bit",
        pipeline_kwargs={"max_tokens": 2048, "temp": 0, "repetition_penalty": 1.2},
    )

    # Setup verbose mode: https://stackoverflow.com/a/77629872/10914628
    model = ChatMLX(llm=llm, verbose=True).with_config(
        {"callbacks": [ConsoleCallbackHandler()]}
    )

    predictable_model = ChatMLX(llm=predictable_llm)
    print("Model loaded.")

else:
    if "FIREWORKS_API_KEY" not in os.environ:
        os.environ["FIREWORKS_API_KEY"] = getpass.getpass(
            "Enter your Fireworks API key: "
        )

    print("Loading Fireworks model...")
    model = ChatFireworks(
        model="accounts/fireworks/models/llama-v3-70b-instruct",
        temperature=0.6,
        max_tokens=4096,
        timeout=None,
        max_retries=0,
        # other params...
    )

    predictable_model = ChatFireworks(
        model="accounts/fireworks/models/llama-v3p1-8b-instruct",
        temperature=0,
        max_tokens=256,
        timeout=None,
        max_retries=0,
        # other params...
    )
    print("Fireworks model loaded.")

Loading Fireworks model...
Fireworks model loaded.


In [6]:
from owlready2 import *

onto = get_ontology("file://worlds/story_world_zombies.rdf").load()

# Verify that the story world is internally coherent, at least by OWL standards
# with onto:
# sync_reasoner()

# for l in onto.CurrentLocation.instances(): print(f"CurrentLocation: {l.hasName}")
# for c in onto.Character.instances(): print(c.hasName)

# print(onto.Player.instances()[0].INDIRECT_isLocatedAt)

Issue: The reasoner will use the state of the ontology at the moment it is instantiated.
If the player moves to a new location, the reasoner will not be updated with this new information.

Source: https://dice-group.github.io/owlapy/usage/reasoning_details.html

Solution: Use isolated worlds, one that is clean, which we update and change with asserted facts, the second one that is used for inference.
When the player moves, we change the clean world, and use it for inference.


In [7]:
from generator.utils import (
    encode_entity_name,
    decode_entity_name,
    find_levenshtein_match,
)

# Conversation loop
So, we have pre-generated the first AI message, which lays out the start of the game.

Then, all subsequent turns will use the same system prompt, as well as dynamically enhancing the player's message with relevant information from the KG

After the player sends their message, we either can:

- Have the LLM respond without knowledge, then analyze the message pair for changes to the KG
- Have the LLM analyze the user's message for intent (especially moving to new locations, which will require the information of the new location to generate the response)

I think the second approach is just better, especially in the case of moving to new locations, which change the knowledge necessary to generate the response.



Once the AI response is generated, we also want to extract changes to the world from the message pair, based on the AI response (item lost, item gained)


So... Basically we need to do intent analysis of the Human message.


Let's summarize a typical conversation turn:

0. The player reads the game's message and decides on something to do.
1. The player's message is analyzed with a prompt, made to extract actionable intent that requires new knowledge (move to other location)
2. If the player moves to another location, we need to retrieve the information about this new location and use that info to generate the AI response. At the same time, we update the KG (player's position)
3. Generate the game response
4. While the player reads and decides on their next turn and types it, analyze the last (message, response) pair for changes to the world: updates to the player's inventory (remove, add or change item's description), updates to characters.

## Move intent
Immediately after the player's message, we need to know if we need to load information about a different location, in order to guide the generation of the response.

In [9]:
INTENT_ANALYSIS_TEMPLATE = """
Human: Please analyze the player's message and determine if they **explicitly** intend to move to a nearby location **immediately**.
Nearby locations: {{nearby_locations_names}}
Player message: "{{player_message}}"

**Please output only the *exact location name* or 'none' with ABSOLUTELY no additional explanation, notes or details. If it doesn't fit, output 'none'**
Output:
"""

In [10]:
from langchain_core.prompts import PromptTemplate


def extract_move_intent(model, message: str, onto) -> Optional[onto.Location]:
    prompt = PromptTemplate(template=INTENT_ANALYSIS_TEMPLATE, template_format="jinja2")

    with onto:
        nearby_locations = onto.Player.instances()[
            0
        ].INDIRECT_isLocatedAt.INDIRECT_isLinkedToLocation

        nearby_locations_names = []
        for l in nearby_locations:
            if l.name == "CurrentLocation":
                continue

            nearby_locations_names.append(l.hasName)

        chain = prompt | model

        analysis = chain.invoke(
            {
                "nearby_locations_names": ", ".join(
                    f'"{l}"' for l in nearby_locations_names
                ),
                "player_message": message,
                "language": language,
            }
        )
        # print(f"Output: {analysis.content}")

        result = analysis.content.strip().strip('"')

        if result.lower() == "none":
            return None

        # print(result)

        entity = find_levenshtein_match(result, onto.Location.instances())

        return entity

In [11]:
print(extract_move_intent(predictable_model, "Aller à la mine", onto))
print(extract_move_intent(predictable_model, "Aller au bunker", onto))
print(extract_move_intent(predictable_model, "Manger une pomme", onto))

None
None
None


## Response generation

In [16]:
CHAT_SYSTEM_PROMPT = """
You are an LLM designed to act as the engine for a text adventure game set in "{{setting}}".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large. You should discreetly tell the player they can go there, without being too explicit.
If the player has spent a long time in a location, you can push them a little more explicitly to move to different locations.
The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.
Do not reveal character names unless the player knows them.

Your messages should be short. Please do not produce lengthy messages. Your messages should be one to two sentences long. The player can always ask for more details ! For dialogues, you will output only one line of dialogue, and let the player respond.

Use the game elements provided:

# Game state
The player's current location is "{{location.hasName}}". {{location.hasName}} is described as "{{location.hasDescription}}".

The locations accessible from where the player is are {{nearby_locations_names}}. You should discreetly tell the player they can go there, without being too explicit.

The player's character is named "{{player.hasName}}" and is described as "{{player.hasDescription}}".
The player's goal is "{{player.hasGoal[0].hasDescription}}"

{%- if characters_nearby %}
Characters present in {{location.hasName}}:
{%- for character in characters_nearby %}
    - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
{%- endfor %}
{%- else %}
    There are no other characters present in {{location.hasName}}.
{%- endif %}

{%- if items_nearby %}
Items present in {{location.hasName}}:
{%- for item in items_nearby %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}
{%- else %}
    There are no items present in {{location.hasName}}.
{%- endif %}

{%- if player.INDIRECT_ownsItem %}
Items in the player's inventory:
{%- for item in player.INDIRECT_ownsItem %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}
{%- else %}
    The player has no items in their inventory.
{%- endif %}

{%- if move_intent_location %}
The parser has detected that the player intends to move to {{move_intent_location.hasName}}.
    {{move_intent_location.hasName}} is described as "{{move_intent_location.hasDescription}}".
    {%- if move_intent_location.INDIRECT_containsCharacter %}
        Characters present in {{move_intent_location.hasName}}:
        {%- for character in move_intent_location.INDIRECT_containsCharacter %}
            - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
        {%- endfor %}
    {%- endif %}
    {%- if move_intent_location.INDIRECT_containsItem %}
        Items present in {{move_intent_location.hasName}}:
        {%- for item in move_intent_location.INDIRECT_containsItem %}
            - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
        {%- endfor %}
    {%- endif %}

If you confirm the move, add the token "<CONFIRM_MOVE>" to your response.
**Keep in mind that the player will read your response before typing theirs, and only the token will be removed from your response.**
{%- endif %}

Always answer in {{language}}
""".strip()

### Create a conversation history

In [84]:
# https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html
from typing import Optional, List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [93]:
async def converse(message: str):
    # 1. Extract move intent
    move_intent_location = extract_move_intent(predictable_model, message, onto)

    # 2. Retrieve relevant information from the KG
    # 2.1 If the player intends to move, add the new location information to the chat prompt
    if move_intent_location:
        print(f"Move intent: {move_intent_location}")

    chat_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", CHAT_SYSTEM_PROMPT),
            ("ai", first_message),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{{message}}"),
        ],
        template_format="jinja2",
    )

    chain = chat_prompt | model

    chain_with_history = RunnableWithMessageHistory(
        chain,
        get_by_session_id,
        input_messages_key="message",
        history_messages_key="history",
    )

    # Retrieve relevant information from the KG
    with onto:
        player = list(onto.Player.instances())[0]
        current_location = player.INDIRECT_isLocatedAt
        characters_nearby = current_location.INDIRECT_containsCharacter
        locations_nearby = current_location.INDIRECT_isLinkedToLocation
        nearby_locations_names = [l.hasName for l in locations_nearby]
        items_nearby = current_location.INDIRECT_containsItem

        parameters = {
            "setting": setting,
            "language": language,
            "location": current_location,
            "move_intent_location": move_intent_location,
            "characters_nearby": characters_nearby,
            "items_nearby": items_nearby,
            "player": player,
            "nearby_locations_names": nearby_locations_names,
            "message": message,
        }

        response = await chain_with_history.ainvoke(
            parameters, config={"configurable": {"session_id": "foo"}}
        )

        # Find if the LLM accepted the move, and filter out the <CONFIRM_MOVE> token
        if "<CONFIRM_MOVE>" in response.content:
            print("Move confirmed by the LLM")
            response = response.replace("<CONFIRM_MOVE>", "").strip()

            # If the LLM accepted the move, update the KG
            with onto:
                for follower in player.INDIRECT_hasFollower:
                    follower.isLocatedAt = move_intent_location
                player.isLocatedAt = move_intent_location

        inventory_actions = extract_inventory_actions(
            predictable_model, message, response.content, onto
        )
        character_actions = extract_character_actions(
            predictable_model, message, response.content, onto
        )
        apply_inventory_actions(inventory_actions.get("actions", []), onto)
        apply_character_actions(character_actions.get("actions", []), onto)
    return response

After the response, we need to report any change to the world state into the knowledge graph.

A player needs to be able to interact with the graph in a few ways:
- move to a nearby location (handled in pre-response generation step)
- claim ownership (of an unowned item): take, buy
- change ownership (of an owned item): give, drop, sell
- change in emotional state of any character
- change in relationship between characters
- add or remove a follower of the player
- update item description (for example if an item deteriorates, etc)
- update character description (for example if a character needs to remember something)
- update location description (for example if the player changed the location in some notable way)
- trigger event
- update events memory (?)
- death of a character (?)

But these are a lot of tasks to ask from one prompt to a small model like Llama3.1-8B-8bit

We can classify these tasks in groups:
- Inventory related tasks
- Character related tasks
- Location related tasks

Now, we can imagine using a prompt for each one of these categories, in order to get a fair balance of speed and accuracy

In [77]:
INVENTORY_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and determine if they intend to interact with an item.

# Action Types
- "create" if you need to create a new item that doesn't exist yet.
- "destroy" if an item should be removed from the game.
- "claim" if the player has taken or picked up an unowned item.
- "give" if the player has given an item they own to another character. In this case, you will provide the name of the character the player gives the item to. **If the character doesn't exist, ignore the action.**
- "drop" if the player has dropped an item they own in the current location.
- "alter" if the player's actions have changed the state of an item. In this case, you will provide an updated description of the item.

{%- if player.INDIRECT_ownsItem %}
# Possible items whose state to change
**Owned items**:
{%- for item in player.INDIRECT_ownsItem %}
- {{item.hasName}}: {{item.hasDescription}}
{%- endfor %}
{%- else %}
There are no items in the player's inventory.
{%- endif %}

{%- if nearby_items %}
**Unowned items in the current location**:
{%- for item in nearby_items %}
{%- if not item.INDIRECT_isOwnedBy %}
- {{item.hasName}}: {{item.hasDescription}}
{%- endif %}
{%- endfor %}
{%- else %}
There are no unowned items in the current location.
{%- endif %}

{%- if nearby_characters %}
**Names of the characters present in the current location**:
{%- for character in nearby_characters %}
    {%- if character.hasName != player.hasName %}
- "{{character.hasName}}"
    {%- endif %}
{%- endfor %}
{%- else %}
There are no other characters present in the current location.
{%- endif %}

# Last message from the player and game response
Player message: "{{player_message}}"
Game response: "{{game_response}}"

**Please output a list of actions that the game system needs to perform to update the game state, in the following JSON format:**
If none of these fit, output an empty list.

```json
{
    "actions": [
        {"action": "create", "item": "map", "description": "A map of the area."},
        {"action": "claim", "item": "map"},
        {"action": "give", "item": "coin", "target": "John Brown"},
        {"action": "drop", "item": "bucket"},
        {"action": "destroy", "item": "apple"},
        {"action": "alter", "item": "sword", "description": "The sword is now covered in rust."}
    ]
}
```

Output:
"""

In [78]:
def extract_inventory_actions(model, message: str, game_response: str, onto):
    prompt = PromptTemplate(
        template=INVENTORY_ACTIONS_PARSER_TEMPLATE, template_format="jinja2"
    )

    parser = JsonOutputParser()

    chain = prompt | model | parser

    with onto:
        player = onto.Player.instances()[0]
        current_location = player.INDIRECT_isLocatedAt

        parameters = {
            "player_message": message,
            "game_response": game_response,
            "player": player,
            "onto": onto,
            "nearby_characters": current_location.INDIRECT_containsCharacter,
            "nearby_items": current_location.INDIRECT_containsItem,
        }

        # Print the prompt to inspect it
        print(prompt.invoke(parameters).text)

        response = chain.invoke(parameters)

    return response

In [60]:
print(
    extract_inventory_actions(
        predictable_model, "Je prends la carte", "Vous prenez la carte.", onto
    )
)


Human: Please analyze the player's message and determine if they intend to interact with an item.

# Action Types
- "create" if you need to create a new item that doesn't exist yet.
- "destroy" if an item should be removed from the game.
- "claim" if the player has taken or picked up an unowned item.
- "give" if the player has given an item they own to another character. In this case, you will provide the name of the character the player gives the item to. **If the character doesn't exist, ignore the action.**
- "drop" if the player has dropped an item they own in the current location.
- "alter" if the player's actions have changed the state of an item. In this case, you will provide an updated description of the item.
# Possible items whose state to change
**Owned items**:
- Eau de vie: Une bouteille d'eau de vie qui peut être utilisée pour se soigner ou comme moyen d'échange.
- Couteau de poche: Un petit couteau utile pour se défendre et couper des cordes ou des tissus.
- Kit de soi

In [91]:
def apply_inventory_actions(actions: list, onto):
    with onto:
        player = onto.Player.instances()[0]
        for action in actions:
            if action["action"] == "create":
                item_name = action["item"]
                item = onto.Item(encode_entity_name(item_name))
                item.hasDescription = action["description"]
                player.ownsItem.append(item)
            elif action["action"] == "destroy":
                item = find_levenshtein_match(action["item"], onto.Item.instances())
                player.ownsItem.remove(item)
                destroy_entity(item)
            elif action["action"] == "claim":
                item = find_levenshtein_match(action["item"], onto.Item.instances())
                player.ownsItem.append(item)
                item.itemIsLocatedAt = None
            elif action["action"] == "give":
                item = find_levenshtein_match(action["item"], onto.Item.instances())
                target = find_levenshtein_match(
                    action["target"], onto.Character.instances()
                )
                player.ownsItem.remove(item)
                item.isOwnedBy = target
            elif action["action"] == "drop":
                item = find_levenshtein_match(action["item"], onto.Item.instances())
                player.ownsItem.remove(item)
                item.itemIsLocatedAt = current_location
            elif action["action"] == "alter":
                item = find_levenshtein_match(action["item"], onto.Item.instances())
                item.hasDescription = action["description"]

In [80]:
CHARACTER_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and corresponding game response to determine if the interaction changed something about the characters present in the scene

**Action Types**:.
- "start_following" if a character has started following the player.
- "stop_following" if a character has stopped following the player.
- "change_health" if a character's health has changed. Add a new description of the character's health.
- "change_description" if a character's description needs to be changed. Add a new description of the character.
- "become_enemies" if two characters have become enemies. (symmetrical)
- "become_friends" if two characters have become friends. (symmetrical)
- "become_rivals" if two characters have become rivals. (symmetrical)
- "become_neutral" if two characters have become neutral. (symmetrical)
- "give_allegiance" if a character has given allegiance to another character. (asymmetrical)
- "now_knows" if a character now knows about another character's existence. (asymmetrical)
- "fall_in_love" if a character has fallen in love with another character. (asymmetrical)

Most relationships are symmetrical, but some are not, like loves. If love is reciprocal, output two actions, one for each character.

The player is named "{{player.hasName}}" and is described as "{{player.hasDescription}}".

**Characters present**:
{%- for character in characters_present %}
- {{character.hasName}}: {{character.hasDescription}}
    {%- if character.INDIRECT_knows %}knows: {%- for known_character in character.INDIRECT_knows %} "{{known_character.hasName}}" {%- endfor %}{%- endif %}
    {%- if character.INDIRECT_isEnemyWith %}is enemy with: {%- for enemy in character.INDIRECT_isEnemyWith %} "{{enemy.hasName}}" {%- endfor %}{%- endif %}
    {%- if character.INDIRECT_hasFriendshipWith %}is friend with: {%- for friend in character.INDIRECT_hasFriendshipWith %} "{{friend.hasName}}" {%- endfor %}{%- endif %}
    {%- if character.INDIRECT_isRivalWith %}is rival with: {%- for rival in character.INDIRECT_isRivalWith %} "{{rival.hasName}}" {%- endfor %}{%- endif %}
    {%- if character.INDIRECT_loves %}is in love with: {%- for love in character.INDIRECT_loves %} "{{love.hasName}}" {%- endfor %}{%- endif %}
    {%- if character.INDIRECT_hasAllegiance %}has allegiance to: {{character.INDIRECT_hasAllegiance.hasName}}{%- endif %}
{%- endfor %}

Player message: "{{player_message}}"
Game response: "{{game_response}}"

**Please output a list of actions that the game system needs to perform to update the game state, in the following JSON format:**
If none of these fit, output an empty list.

```json
{
    "actions": [
        {"action": "become_enemies", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "become_friends", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "become_neutral", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "fall_in_love", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "give_allegiance", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "now_knows", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "start_following", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "stop_following", "subject": "John Brown", "object": "Jane Doe"},
        {"action": "change_health", "subject": "John Brown", "description": "John Brown is now in dire condition. He might die soon."},
        {"action": "change_description", "subject": "John Brown", "description": "John Brown is blablabla (keep parts of the original description) and is now wearing a red hat."}
    ]
}
```

Output:
"""

In [81]:
def extract_character_actions(model, message: str, game_response: str, onto):
    prompt = PromptTemplate(
        template=CHARACTER_ACTIONS_PARSER_TEMPLATE, template_format="jinja2"
    )

    parser = JsonOutputParser()

    chain = prompt | model | parser

    with onto:
        player = onto.Player.instances()[0]
        current_location = player.INDIRECT_isLocatedAt

        parameters = {
            "player_message": message,
            "game_response": game_response,
            "player": player,
            "onto": onto,
            "characters_present": current_location.INDIRECT_containsCharacter,
        }

        # Print the prompt to inspect it
        print(prompt.invoke(parameters).text)

        response = chain.invoke(parameters)

    return response

In [89]:
print(
    extract_character_actions(
        predictable_model,
        "Jinspecte mon inventaire",
        "Votre inventaire est vide.",
        onto,
    )
)


Human: Please analyze the player's message and corresponding game response to determine if the interaction changed something about the characters present in the scene

**Action Types**:.
- "start_following" if a character has started following the player.
- "stop_following" if a character has stopped following the player.
- "change_health" if a character's health has changed. Add a new description of the character's health.
- "change_description" if a character's description needs to be changed. Add a new description of the character.
- "become_enemies" if two characters have become enemies. (symmetrical)
- "become_friends" if two characters have become friends. (symmetrical)
- "become_rivals" if two characters have become rivals. (symmetrical)
- "become_neutral" if two characters have become neutral. (symmetrical)
- "give_allegiance" if a character has given allegiance to another character. (asymmetrical)
- "now_knows" if a character now knows about another character's existence. (as

In [87]:
def apply_character_actions(actions: list, onto):
    with onto:
        player = onto.Player.instances()[0]
        for action in actions:
            print(action)
            if action["action"] == "start_following":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                if subject:
                    player.hasFollower.append(subject)
            elif action["action"] == "stop_following":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                if subject:
                    player.hasFollower.remove(subject)
            elif action["action"] == "change_health":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                if subject:
                    subject.hasHealth = action["description"]
            elif action["action"] == "change_description":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                if subject:
                    subject.hasDescription = action["description"]
            elif action["action"] == "become_enemies":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                _object = find_levenshtein_match(
                    action["object"], onto.Character.instances()
                )
                if subject and _object:
                    subject.isEnemyWith.append(_object)
                    _object.isEnemyWith.append(subject)
            elif action["action"] == "become_friends":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                _object = find_levenshtein_match(
                    action["object"], onto.Character.instances()
                )
                if subject and _object:
                    subject.hasFriendshipWith.append(_object)
                    _object.hasFriendshipWith.append(subject)
            elif action["action"] == "become_neutral":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                _object = find_levenshtein_match(
                    action["object"], onto.Character.instances()
                )
                if subject and _object:
                    subject.isEnemyWith.remove(_object)
                    subject.isRivalWith.remove(_object)
                    subject.hasFriendshipWith.remove(_object)
                    _object.isEnemyWith.remove(subject)
                    _object.isRivalWith.remove(subject)
                    _object.hasFriendshipWith.remove(subject)
            elif action["action"] == "fall_in_love":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                _object = find_levenshtein_match(
                    action["object"], onto.Character.instances()
                )
                if subject and _object:
                    # Love is sometimes reciprocal, sometimes not
                    subject.loves.append(_object)
            elif action["action"] == "give_allegiance":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                _object = find_levenshtein_match(
                    action["object"], onto.Character.instances()
                )
                if subject and _object:
                    subject.hasAllegiance = _object
            elif action["action"] == "rescind_allegiance":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                if subject:
                    subject.hasAllegiance = None
            elif action["action"] == "now_knows":
                subject = find_levenshtein_match(
                    action["subject"], onto.Character.instances()
                )
                _object = find_levenshtein_match(
                    action["object"], onto.Character.instances()
                )
                if subject and _object:
                    subject.knows.append(_object)

# Play !

In [94]:
set_verbose(False)
# Test the conversation loop
while True:
    message = input("Player: ")
    print(f"Player: {message}")

    # Exit the loop
    if message == "exit":
        break

    print(await converse(message))

Player: J'inspecte mon inventaire

Human: Please analyze the player's message and determine if they intend to interact with an item.

# Action Types
- "create" if you need to create a new item that doesn't exist yet.
- "destroy" if an item should be removed from the game.
- "claim" if the player has taken or picked up an unowned item.
- "give" if the player has given an item they own to another character. In this case, you will provide the name of the character the player gives the item to. **If the character doesn't exist, ignore the action.**
- "drop" if the player has dropped an item they own in the current location.
- "alter" if the player's actions have changed the state of an item. In this case, you will provide an updated description of the item.
# Possible items whose state to change
**Owned items**:
- Eau de vie: Une bouteille d'eau de vie qui peut être utilisée pour se soigner ou comme moyen d'échange.
- Couteau de poche: Un petit couteau utile pour se défendre et couper des 